In [1]:
import re
!pip install transformers torch

In [2]:
import torch
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Function to compute sentence embeddings
def get_sentence_embedding(sentence):
    inputs = tokenizer(sentence, return_tensors='pt', truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).numpy()


C:\Users\viksu\Desktop\Diploma\TopicExtractor\venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [22]:
import spacy

# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

def extract_context(text, topics, window_size=5):
    """
    Extract context of given topics from text using token-based approach.
    
    Parameters:
    - text (str): The input text.
    - topics (dict): Dictionary with topics as keys and lists of keywords as values.
    - window_size (int): Number of tokens to include before and after the topic keyword.
    
    Returns:
    - dict: Dictionary with topics as keys and list of contexts as values.
    """
    # Process the text
    doc = nlp(text)
    tokens = [token.text for token in doc]
    topic_positions = {topic: [] for topic in topics}
    
    for topic, phrases in topics.items():
        for phrase in phrases:
            phrase_tokens = phrase.split()
            n = len(phrase_tokens)
            
            for i in range(len(tokens) - n + 1):
                ngram_tokens = tokens[i:i + n]
                
                if ngram_tokens == phrase_tokens:
                    topic_positions[topic].append(i)
    
    return topic_positions

In [23]:
text = """The month of February offered divergent performance across
the asset classes. While the equity market had a positive
month, mainly driven by a handful of IT, high growth stocks,
(i.e. Nvidia), the bond market ended in negative territory, as
rates’ cuts expectations moved further out into the year.
Strong economic data in the US and a good reporting season,
especially within IT, developed market (DM) equities delivered
a strong positive performance, while China-driven emerging
markets (EM) did even better. Sector-wise, cyclicals clearly
outperformed their more defensive peers, as Cons.
Discretionary, IT and Industrials led the way, while Healthcare,
Cons. Staples and Utilities lagged significantly. In terms of
styles, Quality, Momentum and Growth outperformed Value
and Low Risk. Within fixed income, curves flattened marginally
and yields moved upwards around 30-40 bps on both sides of
the Atlantic. Credit-wise, spreads moved down markedly
across the board, with US investment grade being the only
exception. """

topics = {
    "equity market": ["equity market"],
    "bond": ["bond", "bonds"],
}

contexts = extract_context(text, topics, window_size=5)
for topic, context_list in contexts.items():
    print(f"Contexts for topic '{topic}':")
    for context in context_list:
        print(context)
        print("---")

Contexts for topic 'equity market':
15
---
Contexts for topic 'bond':
42
---
